# Log any table with `mlflow.log_table()`

The `mlflow.evaluate()` feature provides a very useful tool for evaluating mlflow models on specified datasets and logging the metrics and artifacts to MLFlow tracking.

`mlflow.log_table()` provides a flexible option for logging arbitrary tables in cases where `mlflow.evaluate()` isn't the right fit. For example, you may want to test an LLM on an evaluation dataset and log the results without saving/loading it as an MLFlow model.

This example shows how to log a table of model outputs to MLFlow tracking without logging the model or invoking `mlflow.evaluate`.

In [0]:
!pip install --upgrade torch accelerate xformers
!pip install triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir#subdirectory=python

def is_databricks():
    try:
        dbutils
        return True
    except NameError:
        return False
  
if is_databricks():
  dbutils.library.restartPython()

  

In [0]:
import pandas as pd
import mlflow
import torch
import transformers
from transformers import AutoTokenizer, pipeline
import os

def is_databricks():
    try:
        dbutils
        return True
    except NameError:
        return False

if is_databricks():
    os.environ["TRANSFORMERS_CACHE"] = "/dbfs/<location>"
else:
    load_dotenv()

## Load a Hugging Face Model

In [0]:

name = 'mosaicml/mpt-7b-instruct'

config = transformers.AutoConfig.from_pretrained(name, trust_remote_code=True)
config.attn_config['attn_impl'] = 'triton'
config.init_device = 'cuda:0' # For fast initialization directly on GPU!

model = transformers.AutoModelForCausalLM.from_pretrained(
  name,
  config=config,
  torch_dtype=torch.bfloat16, # Load model weights in bfloat16
  trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device='cuda:0')

In [0]:
def format_prompt(instruction):
  INSTRUCTION_KEY = "### Instruction:"
  RESPONSE_KEY = "### Response:"
  INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  PROMPT_FOR_GENERATION_FORMAT = """{intro}
  {instruction_key}
  {instruction}
  {response_key}
  """.format(
      intro=INTRO_BLURB,
      instruction_key=INSTRUCTION_KEY,
      instruction=instruction,
      response_key=RESPONSE_KEY,
  )

  return PROMPT_FOR_GENERATION_FORMAT

In [0]:
# Test the pipeline
with torch.autocast('cuda', dtype=torch.bfloat16):
    print(
        pipe(format_prompt('What is the airspeed velocity of an unladen swallow?'),
            max_new_tokens=100,
            do_sample=True,
            top_k = 10,
            temperature = 1.5,
            use_cache=True,
            return_full_text=False,
            eos_token_id = tokenizer.eos_token_id, 
            pad_token_id = tokenizer.eos_token_id))

## Create an Evaluation Dataset
We want to compare the outputs of a few different prompts with a few different generation parameters.